In [ ]:
import numpy as np
import pandas as pd
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as pyplot
import tensorflow.keras as keras
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout
%matplotlib inline
file=r"C:\Users\VISWAM\Downloads\amf.csv"
db=pd.read_csv(file)
#Converting SMILES to ECFP values 
def get_fingerprint(smiles,size=8192):
  if ((smiles is None) or (pd.isnull(smiles))):
    return np.zeros((size,))
  molecule = Chem.MolFromSmiles(smiles)
  if molecule is None:
    return np.zeros((size,))
  fingerprint = AllChem.GetMorganFingerprintAsBitVect(
      molecule, 2, size)
  arr = np.zeros((1,))
  DataStructs.ConvertToNumpyArray(fingerprint, arr)
  return arr 
cols=db.columns[[0,2,4,6,8,12]]
col=db.columns[[1,3,5,7]]
array=[]
for a in cols:
    j=[]
    for i in range(0,len(db[a])):
     temp=(get_fingerprint(db[a][i])).astype(int)
     j.append(temp)
    db[a]=j
db.shape
db.fillna(0,inplace=True)
db1=pd.concat([pd.DataFrame(db["R"].values.tolist()), pd.DataFrame(db["r1"].values.tolist()),pd.DataFrame(db["r2"].values.tolist()),pd.DataFrame(db["c1"].values.tolist()),db['c1c'],pd.DataFrame(db["c2"].values.tolist()),db['c2c'],db['t'],db['T']], axis=1)
y=pd.factorize(db["class"])[0]
cn=['Small Group','Large Group', 'Anti Markovnokov','Markovnikov']
y = to_categorical(y)
def f1(y_true, y_pred):
    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)
    return 2 * (K.sum(y_true * y_pred)+ K.epsilon()) / (K.sum(y_true) + K.sum(y_pred) + K.epsilon())

In [ ]:
#Stratified kfold validation with k=10
train=[]
test=[]
trainf=[]
testf=[]
skf = StratifiedKFold(n_splits=10,random_state=32,shuffle=True)
for tr,te in skf.split(db1, db['class']):
    classifier = Sequential()
    reg = tf.keras.regularizers.l1_l2(l1=0.00005, l2=0.004)
    classifier.add(Dense(512, activation='relu', kernel_initializer='random_normal', input_dim=40964,kernel_regularizer=reg))
    classifier.add(Dropout(0.5))
    classifier.add(Dense(512, activation='relu', kernel_initializer='random_normal',kernel_regularizer=reg))
    classifier.add(Dropout(0.5))
    classifier.add(Dense(512, activation='relu', kernel_initializer='random_normal',kernel_regularizer=reg))
    classifier.add(Dropout(0.5))
    classifier.add(Dense(4, activation='softmax', kernel_initializer='random_normal'))
    classifier.compile(optimizer ='adam',loss='categorical_crossentropy', metrics=['accuracy'])
    history = classifier.fit(db1.iloc[tr], y[tr], validation_data=(db1.iloc[te], y[te]), epochs=75, verbose=2)
# evaluate the model
    _, train_acc = classifier.evaluate(db1.iloc[tr], y[tr], verbose=0)
    _, test_acc = classifier.evaluate(db1.iloc[te], y[te], verbose=0)
    print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))
    train.append(train_acc)
    test.append(test_acc)
    yp=classifier.predict(db1.iloc[te])
    vyp=classifier.predict(db1.iloc[tr])
    trainf.append(f1(y[tr],vyp))
    testf.append(f1(y[te],yp))

In [ ]:
#plotting validation accuracy score
n=np.array([1,2,3,4,5,6,7,8,9,10])
te=np.transpose(test)
tr=np.transpose(train)
fig=pyplot.figure()
pyplot.figure(figsize=(10,6))
pyplot.title("Stratified Kfold")
pyplot.bar(n,tr*100,.4,label='Train set')
pyplot.bar(n+.3,te*100,.4,label='Test set')
pyplot.xticks(np.arange(len(n)+1))
pyplot.xlabel("Iteration number")
pyplot.ylabel("Accuracy")
pyplot.ylim(80,101)
pyplot.legend(bbox_to_anchor=(1,0), loc="lower right", 
                bbox_transform=fig.transFigure, ncol=2)
pyplot.savefig('Classification1.png')
pyplot.show()
print(np.mean(train),np.mean(test))

In [ ]:
#plotting validation F1-score
tef=np.transpose(testf)
trf=np.transpose(trainf)
fig=pyplot.figure()
pyplot.figure(figsize=(10,6))
pyplot.title("Stratified Kfold")
pyplot.bar(n,trf,.4,label='Train set')
pyplot.bar(n+.3,tef,.4,label='Test set')
pyplot.xticks(np.arange(len(n)+1))
pyplot.xlabel("Iteration number")
pyplot.ylabel("F1 score")
pyplot.ylim(.88,.99)
pyplot.legend(bbox_to_anchor=(1,0), loc="lower right", 
                bbox_transform=fig.transFigure, ncol=2)
pyplot.savefig('Classification2.png')
pyplot.show()
print(np.mean(trainf),np.mean(testf))